## ReMKiT1D input generator - flux limiters with calculation trees and CVODE

This notebook generates an input file for a ReMKiT1D run solving a simple advection problem by building a MUSCL scheme using built-in unary transformations and stationary model evaluations. It demonstrates several features new to ReMKiT1D v1.2.0:
- Use of built-in flux limiter (slope limiter) functions within `UnaryTransform`.
- Support for the SUNDIALS `CVODE` integrator. Basic functionality is provided in this example.

In [ ]:
from RMK_support import RKWrapper ,Grid
import RMK_support.simple_containers as sc
import RMK_support.common_models as cm
import RMK_support.IO_support as io
from RMK_support import Node, treeDerivation, UnaryTransform

import numpy as np
import holoviews as hv
import matplotlib.pyplot as plt

Standard setup of the wrapper, with uniform periodic spatial grid

In [ ]:
#Wrapper initialization
rk = RKWrapper()

#I/O setup
rk.jsonFilepath = "./config.json" 
hdf5Filepath = "./RMKOutput/RMK_limiters_CVODE/"
rk.setHDF5Path(hdf5Filepath) 

#MPI setup
rk.setMPIData(numProcsX=4)

#Grid initialization
xGridWidths = 0.025*np.ones(512)
gridObj = Grid(xGridWidths, interpretXGridAsWidths=True,isPeriodic=True)
rk.grid = gridObj

In [ ]:
#Variables

# Generate a steep-sided "box" of finite value in 1D space:
n = np.ones(512) # Try np.zeros (CVODE might struggle, but RK3 should work fine)
n[100:200] = 2

# Define density variables for three different scenarios:
rk.addVarAndDual('n_s',n,isCommunicated=True) # Flux limiter: superbee
rk.addVarAndDual('n_m',n,isCommunicated=True) # Flux limiter: minmod
rk.addVarAndDual('n_u',n,isCommunicated=True) # Unlimited flux

#Note that time is not added here, in from v1.2.0 time is added automatically unless the wrapper is instructed otherwise

### Building a MUSCL scheme from scratch 

We wish to calculate a Rusanov (Lax-Friedrichs) flux for the simple advection equation 

$$ \frac{\partial n}{\partial t} + \frac{\partial n}{\partial x} = 0 $$ 
which has the flux $F=n$ and the flux Jacobian $\partial F/ \partial n = 1$.

On a cell edge, the numerical flux for this problem (simplified because of the trivial Jacobian) is given by 

$$ F^*_{i+1/2} = \langle F \rangle_{i+1/2} - [\![ n ]\!]_{i+1/2}$$

where the average and jump operators are

$$ \langle F \rangle_{i+1/2} = \frac{1}{2}\left[n^R_{i+1/2}+n^L_{i+1/2}\right] $$
$$ [\![ n ]\!]_{i+1/2} = \frac{1}{2}\left[n^R_{i+1/2}-n^L_{i+1/2}\right] $$

The two values $n^R$ and $n^L$ are the right and left values of the advected quantity at the cell edge, given by

$$n^L_{i+1/2} = n_i + 0.5\phi(r_i)(n_{i+1}-n_{i})$$
$$n^R_{i+1/2} = n_{i+1} - 0.5\phi(r_{i+1})(n_{i+2}-n_{i+1})$$
where 

$$r_i = \frac{n_i-n_{i-1}}{n_{i+1}-n_{i}}$$
is the slope ratio and $\phi$ is the limiter function we wish to use. In this example, we will use the minmod and superbee limiters whose respective functions are:

$$ \phi_m(r) = \max[0,\min[0,r]] $$
$$ \phi_s(r) = \max[0,\min[2r,1],\min[r,2]]$$

### Adding the slopes and limiters

We will use the calculation tree feature in ReMKiT1D, together with several built-in unary transforms to do this.

The `slopeRatio` unary transform calculates

$$r_i = \frac{n_i-n_{i-k}}{n_{i+k}-n_{i}}$$
where $n_{i\pm k}$ is calculating by shifting the MPI local array left or right (more on the unary `shift` below). 
The shift amount k is determined by the first integer parameter passed to the uniform transform. To handle the case when the denominator is small, we introduce two real parameters $a_1$ and $a_2$, so that if the denominator is less than $a_1$

$$r_i = 1, n_{i+k}-2n_i + n_{i-k} < a_1$$
otherwise 
$$r_i = a_2  \text{sgn}(n_i-n_{i-k}) \text{sgn}(n_i-n_{i-k})$$

In [ ]:
#Here we take consecutive slopes, setting the small a1 to 1e-6 and the large a2 to 1e2
slopeRatio = UnaryTransform("slopeRatio",realParams=[1e-6,1e2],intParams=[1])

# Now we add the two variables containing the slope ratios 

rk.addVar("slopeRatio_s",isDerived=True,derivationRule=sc.derivationRule("slRatio_s",["n_s"]),derivOptions=treeDerivation(slopeRatio(Node("n_s"))))

rk.addVar("slopeRatio_m",isDerived=True,derivationRule=sc.derivationRule("slRatio_m",["n_m"]),derivOptions=treeDerivation(slopeRatio(Node("n_m"))))

The limiters are also unary transform we can easily use to build the limiter variables

In [ ]:
superbee=UnaryTransform("superbeeLimiter")

#We fold the 0.5 into the limiter variable 
rk.addVar("densLimiter_s",isDerived=True,derivationRule=sc.derivationRule("densLimiter_s",["slopeRatio_s"]),derivOptions=treeDerivation(superbee(Node("slopeRatio_s"))/2),isOnDualGrid=True,isCommunicated=True)

minmod=UnaryTransform("minmodLimiter")
rk.addVar("densLimiter_m",isDerived=True,derivationRule=sc.derivationRule("densLimiter_m",["slopeRatio_m"]),derivOptions=treeDerivation(minmod(Node("slopeRatio_m"))/2),isOnDualGrid=True,isCommunicated=True)

# The limiters are communicated because we need the phi(r_i+1) for the left edge densities


### The shift transform

The `shift` unary transform allows us to perform low level array manipulations using calculation trees. Specifically, shift is parameterized by a single integer, so that

$$\left[\text{shift}_k(n)\right]_i = n_{i-k}$$
with the shift performed cyclically on the local ReMKiT1D arrays (at each processor), including the halo values. Care should be exercised near boundaries, as no transform in ReMKiT1D has any global information.

We use the shift transform when we want to perform finite difference operations within calculation trees.

### The average and jump operators

We will use a `customFluid1DStencil` to build the average and jump operators. For this, we will need to write them out explicitly first, resulting in 

$$ \langle n \rangle_{i+1/2} = \frac{1}{2}\left[n_{i+1} - 0.5\phi(r_{i+1})(n_{i+2}-n_{i+1}) + n_i + 0.5\phi(r_i)(n_{i+1}-n_{i})\right] $$
$$ [\![ n ]\!]_{i+1/2} = \frac{1}{2}\left[n_{i+1} - 0.5\phi(r_{i+1})(n_{i+2}-n_{i+1}) - n_i - 0.5\phi(r_i)(n_{i+1}-n_{i})\right] $$

We notice that both operators can be written in the form 

$$v_0 n_i + v_1 n_{i+1} + v_2 n_{i+2}$$
where the $v$'s are functions only of $\phi$. We introduce $\Phi=0.5\phi$ to simplify notation and to reflect the limiter variable we've added having the $0.5$ factor included.

Let's see what $v_1$ is for the jump operator. We collect all terms multiplying $n_{i+1}$ on the RHS, and get 

$$\left[v_{jump,1}\right]_i = 1 - \Phi_i + \Phi_{i+1}$$

To write the above in calculation tree form, we use the `shift` operator with $k=-1$

$$v_{jump,1} = 1 - \Phi + \text{shift}_{-1}(\Phi)$$

We can similarly find and write all of the $v$ variables, and we add them below.

In [ ]:
# Setting the shift operator for use
shift = UnaryTransform("shift",intParams=[-1])

# The superbee v's
rk.addVar("leftVarAvg_s",isDerived=True,derivationRule=sc.derivationRule("left_s",["densLimiter_s"]),derivOptions=treeDerivation(1-Node("densLimiter_s")))

rk.addVar("midVarAvg_s",isDerived=True,derivationRule=sc.derivationRule("midAvg_s",["densLimiter_s"]),derivOptions=treeDerivation(1+Node("densLimiter_s")+shift(Node("densLimiter_s"))))

rk.addVar("leftVarJmp_s",isDerived=True,derivationRule=sc.derivationRule("leftJmp_s",["densLimiter_s"]),derivOptions=treeDerivation(-1+Node("densLimiter_s")))

rk.addVar("rightVarAvg_s",isDerived=True,derivationRule=sc.derivationRule("rightAvg_s",["densLimiter_s"]),derivOptions=treeDerivation(-shift(Node("densLimiter_s"))))

rk.addVar("midVarJmp_s",isDerived=True,derivationRule=sc.derivationRule("midJmp_s",["densLimiter_s"]),derivOptions=treeDerivation(1-Node("densLimiter_s")+shift(Node("densLimiter_s"))))

rk.addVar("rightVarJmp_s",isDerived=True,derivationRule=sc.derivationRule("rightJmp_s",["densLimiter_s"]),derivOptions=treeDerivation(-shift(Node("densLimiter_s"))))

#The minmod v's
rk.addVar("leftVarAvg_m",isDerived=True,derivationRule=sc.derivationRule("left_m",["densLimiter_m"]),derivOptions=treeDerivation(1-Node("densLimiter_m")))

rk.addVar("midVarAvg_m",isDerived=True,derivationRule=sc.derivationRule("midAvg_m",["densLimiter_m"]),derivOptions=treeDerivation(1+Node("densLimiter_m")+shift(Node("densLimiter_m"))))

rk.addVar("leftVarJmp_m",isDerived=True,derivationRule=sc.derivationRule("leftJmp_m",["densLimiter_m"]),derivOptions=treeDerivation(-1+Node("densLimiter_m")))

rk.addVar("rightVarAvg_m",isDerived=True,derivationRule=sc.derivationRule("rightAvg_m",["densLimiter_m"]),derivOptions=treeDerivation(-shift(Node("densLimiter_m"))))

rk.addVar("midVarJmp_m",isDerived=True,derivationRule=sc.derivationRule("midJmp_m",["densLimiter_m"]),derivOptions=treeDerivation(1-Node("densLimiter_m")+shift(Node("densLimiter_m"))))

rk.addVar("rightVarJmp_m",isDerived=True,derivationRule=sc.derivationRule("rightJmp_m",["densLimiter_m"]),derivOptions=treeDerivation(-shift(Node("densLimiter_m"))))

If we think of representing the jump and the average operators as matrix-vector products 

$$ M_{ij} n_{j}$$
we see that the $v$'s above are the columns of the matrix $M$ (up to a factor of $1/2$).

We can use the `customFluid1DStencil` below to generate stencil with the appropriate columns

In [ ]:
# The stencil act on $n_i$,$n_{i+1}$, and $n_{i+2}$, so the stencil shorthand is [0,1,2]. We divide each column by 2 using the fixedColumnVecs, and set the variables corresponding to each of the stencil column using varContColumnVars.

# Superbee stencils
averageStencil_s = sc.customFluid1DStencil([0,1,2],fixedColumnVecs=[0.5*np.ones(512)]*3,varContColumnVars=["leftVarAvg_s","midVarAvg_s","rightVarAvg_s"])

jumpStencil_s = sc.customFluid1DStencil([0,1,2],fixedColumnVecs=[0.5*np.ones(512)]*3,varContColumnVars=["leftVarJmp_s","midVarJmp_s","rightVarJmp_s"])

#Minmod stencils
averageStencil_m = sc.customFluid1DStencil([0,1,2],fixedColumnVecs=[0.5*np.ones(512)]*3,varContColumnVars=["leftVarAvg_m","midVarAvg_m","rightVarAvg_m"])

jumpStencil_m = sc.customFluid1DStencil([0,1,2],fixedColumnVecs=[0.5*np.ones(512)]*3,varContColumnVars=["leftVarJmp_m","midVarJmp_m","rightVarJmp_m"])

Define the three flux variables, including the two limited fluxes and the one unlimited flux:

In [ ]:
rk.addVar("numFlux_s",isStationary=True,isOnDualGrid=True,isCommunicated=True)
rk.addVar("numFlux_m",isStationary=True,isOnDualGrid=True,isCommunicated=True)
rk.addVar("unlimitedFlux",isStationary=True,isOnDualGrid=True,isCommunicated=True)

And then we can add three models, one for each numerical flux. The models correspond to solving 

$$ 0 = - F^* + \langle n \rangle - [\![ n ]\!]$$
so they need 3 terms on the RHS.

In [ ]:

# Superbee flux
newModel = sc.CustomModel("numFlux_s")

newModel.addTerm("id",sc.GeneralMatrixTerm("numFlux_s",customNormConst=-1)) # Note that as of v1.2.0 diagonal stencils are the default option, so we do not need to specify it any more

newModel.addTerm("avg",sc.GeneralMatrixTerm("numFlux_s","n_s",stencilData=averageStencil_s))

newModel.addTerm("jmp",sc.GeneralMatrixTerm("numFlux_s","n_s",customNormConst=-1,stencilData=jumpStencil_s))

# Here we use another v1.2.0 feature, which enables marking models as integrable for easier addition to integration steps, as we'll see below
rk.addModel(newModel,isIntegrable=False)

# Minmod flux
newModel = sc.CustomModel("numFlux_m")

newModel.addTerm("id",sc.GeneralMatrixTerm("numFlux_m",customNormConst=-1))

newModel.addTerm("avg",sc.GeneralMatrixTerm("numFlux_m","n_m",stencilData=averageStencil_m))

newModel.addTerm("jmp",sc.GeneralMatrixTerm("numFlux_m","n_m",customNormConst=-1,stencilData=jumpStencil_m))

rk.addModel(newModel,isIntegrable=False)

# Unlimited flux, given solely by: 0 = -F + <n>
newModel = sc.CustomModel("unlimitedFluxModel")

newModel.addTerm("id",sc.GeneralMatrixTerm("unlimitedFlux",customNormConst=-1))

newModel.addTerm("avg",sc.GeneralMatrixTerm("unlimitedFlux","n_u"))

rk.addModel(newModel,isIntegrable=False)

Finally, because we've marked the models above as not integrable, we need another way of evaluating the variables. In v1.2.0, we can evaluate models where the "evolved" stationary variable is only a function of other variables by adding the following manipulator objects.

These will update and evaluate all terms "evolving" a given variable and add the result to the variable. In our case this will calculate the numerical fluxes by evaluating the matrix terms.

This new feature enables the conversion of purely implicit scripts relying on stationary variables to scripts that can safely use CVODE and RK integrators.

In [ ]:
rk.addStationaryEvaluator("numFlux_s")
rk.addStationaryEvaluator("numFlux_m")
rk.addStationaryEvaluator("unlimitedFlux")

It is then a simple matter to use standard staggered advection models with the numerical flux

In [ ]:
rk.addModel(cm.staggeredAdvection("nAdvection_s","n_s","numFlux_s"))
rk.addModel(cm.staggeredAdvection("nAdvection_m","n_m","numFlux_m"))
rk.addModel(cm.staggeredAdvection("nAdvection_u","n_u","unlimitedFlux"))

### Setting up integrators

Below we set up integrator options. We add both a CVODE and an RK3 integrator.

Try each of these out by setting `usedIntegrator` to either `"CVODE"` or `"RK"`:

In [ ]:
rk.addIntegrator("CVODE",sc.CVODEIntegrator(absTol=1e-12))
rk.addIntegrator("RK",sc.rkIntegrator(3))

rk.setIntegratorGlobalData(initialTimestep=0.001) 

usedIntegrator = "RK"
integratorStep = sc.IntegrationStep(usedIntegrator)

for tag in rk.modelTags(integrableOnly=True):
    integratorStep.addModel(tag)

rk.addIntegrationStep("Step"+usedIntegrator,integratorStep.dict())

rk.setFixedNumTimesteps(10000)
rk.setFixedStepOutput(100)
rk.setPETScOptions(active=False) #We can turn off PETSc objects in the code if we're not using them (this should save some memory, though it is likely not significant)

In [ ]:
rk.writeConfigFile()

## Data analysis

In [ ]:
hv.extension('matplotlib')
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150
hv.output(size=150,dpi=150)

In [ ]:
numFiles = 100
loadpath = hdf5Filepath
loadFilenames = [loadpath+f'ReMKiT1DVarOutput_{i}.h5' for i in range(numFiles+1)]
loadedData = io.loadFromHDF5(rk.varCont,filepaths=loadFilenames)
loadedData

We can compare the dissipative behaviour of each of the three scenarios.

In [ ]:
import panel as pn 
import RMK_support.dashboard_support as ds

pn.extension(comms="vscode") # change comms if not using VSCode
dashboard = ds.ReMKiT1DDashboard(loadedData,rk.grid)

Compare the superbee and minmod flux limiters:

In [ ]:
dashboard.fluidMultiComparison(["n_s","n_m"])

Compare the unlimited flux scenario `"n_u"` with the flux-limited scenarios `"n_s"` and `"n_m"`:

In [ ]:
dashboard.fluidMultiComparison(["n_u","n_s","n_m"])